In [22]:
# Importamos las librerías.
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [23]:
# Vamos a crear un conjunto de datos, que tendrá:
# - 7 atributos numéricos.
# - 3 atributos categóricos.
np.random.seed(0)
X_num = np.random.rand(1000, 7)
X_cat = np.vstack([
            np.random.choice(['No', 'Yes'], 1000),
            np.random.choice(['Young', 'Middle', 'Old'], 1000),
            np.random.choice(['Very Low', 'Low', 'Medium', 'High', 'Very High'], 1000),
        ]).T

In [24]:
# Introducimos un 5% de valores nulos de forma aleatoria
X_num[np.random.choice([True, False], (1000, 7), p=[0.05, 0.95])] = np.nan
X_cat[np.random.choice([True, False], (1000, 3), p=[0.05, 0.95])] = np.nan

In [25]:
# Breve EDA para comprobar valores nulos.
df = pd.concat([pd.DataFrame(X_num), pd.DataFrame(X_cat).replace('nan', np.nan)], axis=1)
df.columns = [f"attr{i}" for i in range(10)]
df.isna().sum(axis=0)

attr0    58
attr1    42
attr2    52
attr3    54
attr4    52
attr5    53
attr6    63
attr7    58
attr8    50
attr9    63
dtype: int64

In [26]:
# Vamos a utilizar dos tipos de imputación distinta:
# - Basada en la media para atributos numéricos.
# - Basada en la moda para atributos categóricos.
# Para ello, utilizaremos un ColumnTransformer.
# Ver: https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html

# Partición en entrenamiento y test.
X_train, X_test = train_test_split(df, random_state=0)

# Procesamiento de valores numéricos.
features_num = [f"attr{i}" for i in range(7)]
imputer_num = SimpleImputer(strategy='mean')
pipeline_num = Pipeline(
    steps=[
        ("imputer", imputer_num)
    ]
)

# Procesamiento de valores categóricos.
features_cat = [f"attr{i}" for i in range(7, 10)]
imputer_cat = SimpleImputer(strategy='most_frequent')
pipeline_cat = Pipeline(
    steps=[
        ("imputer", imputer_cat)
    ]
)

# Pre-procesador "global".
# Dependiendo del tipo de columna se aplica una transformación u otra.
processor = ColumnTransformer(
    transformers=[
        ("num", pipeline_num, features_num),
        ("cat", pipeline_cat, features_cat),
    ]
)

# Realizamos la transformación.
X_train_processed = processor.fit_transform(X_train)
X_test_processed = processor.transform(X_test)

In [38]:
# Vamos a comprobar un par de valores nulos.
print(f"El valor {X_train.iloc[747, 4]} ahora vale {X_train_processed[747, 4]}")
print(f"El valor {X_train.iloc[37,9]} ahora vale {X_train_processed[37, 9]}")

El valor nan ahora vale 0.5069408954869667
El valor nan ahora vale Medium
